# Inferring Gene Regulatory Networks with HuMMuS

This tutorial demonstrates how to use ReCoN to infer **Gene Regulatory Networks (GRNs)** from paired single-cell RNA-seq and ATAC-seq data using the HuMMuS methodology.

## What you will learn

1. How to compute individual network layers (RNA, ATAC, TF)
2. How to link layers via bipartite networks (TF→ATAC, ATAC→RNA)
3. How to integrate everything into a multilayer GRN using Random Walk with Restart

```{warning}
**Python 3.10 required!**

GRN inference requires a Python 3.10 environment due to CellOracle and gimmemotifs dependencies.
See the [Installation guide](../recon_explained/get_ready.rst) for setup instructions.
```

---

## Prerequisites

```{note}
**Required packages**

- `bedtools` - for ATAC-seq peak processing (`conda install -c bioconda bedtools`)
- `muon` - for multi-omics data handling (`pip install muon`)
- `circe-py` - for ATAC co-accessibility networks (included with ReCoN)
```

### Check bedtools installation

In [1]:
import os
import shutil

# Check if bedtools is available
if shutil.which("bedtools") is None:
    # Try to find bedtools in common conda locations
    conda_prefix = os.environ.get("CONDA_PREFIX", "")
    possible_paths = [
        f"{conda_prefix}/bin",
        os.path.expanduser("~/miniconda3/bin"),
        os.path.expanduser("~/anaconda3/bin"),
        "/usr/local/bin",
    ]
    for path in possible_paths:
        bedtools_path = os.path.join(path, "bedtools")
        if os.path.exists(bedtools_path):
            os.environ["PATH"] = path + ":" + os.environ["PATH"]
            print(f"Added {path} to PATH")
            break
    
# Verify bedtools is now available
!which bedtools && bedtools --version

/opt/gensoft/exe/bedtools/2.31.1/scripts/bedtools
bedtools v2.31.1


### Import libraries

In [2]:

import hummuspy.loader  # HuMMuS multilayer network utilities
import circe as ci       # ATAC-seq co-accessibility networks
import recon             # ReCoN GRN inference
import recon.infer_grn   # ReCoN GRN inference

/pasteur/appa/homes/rtrimbou/miniconda3/envs/snakemake/envs/recon-grn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/pasteur/appa/homes/rtrimbou/miniconda3/envs/snakemake/envs/recon-grn/lib/python3.10/site-packages/gimmemotifs/config.py:14: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Using celloracle-lite v0.21.0 (lightweight fork for ReCoN/HuMMuS)
2026-01-12 14:14:43,112 - INFO - Using celloracle-lite v0.21.0 (lightweight fork for ReCoN/HuMMuS)
To install the original CellOracle, please check: https://github.com/morris-lab/CellOracle
2026-01-12 14:14:43,113 - INFO - To install the or

In [3]:
import muon as mu   # Multi-omics data handling
import scanpy as sc # Single-cell analysis

---

## Load Multi-omics Data

We use paired scRNA-seq and scATAC-seq data stored in a MuData object.

```{tip}
The example dataset (PBMC 10X) can be downloaded from the [CIRCE benchmark repository](https://github.com/cantinilab/circe_benchmark).
```

In [4]:
# Load paired RNA + ATAC data
mudata = mu.read_h5mu("./data/build_grn_tuto/pbmc10x.h5mu")
print(f"RNA: {mudata.mod['rna'].shape}, ATAC: {mudata.mod['atac'].shape}")

RNA: (9631, 18410), ATAC: (9631, 215676)


In [5]:
# Subset for faster computation (remove for full analysis)
rna = mudata.mod['rna'][:5000, :1000]   # First 1000 genes
atac = mudata.mod['atac'][:5000, :5000] # First 5000 peaks
print(f"Subset - RNA: {rna.shape}, ATAC: {atac.shape}")

Subset - RNA: (5000, 1000), ATAC: (5000, 5000)


---

## Build the Multilayer GRN

The HuMMuS methodology builds a GRN by integrating multiple layers:

| Layer | Description | Method |
|-------|-------------|--------|
| **RNA layer** | Gene co-regulation patterns (which genes vary together) | GRNBoost2 (Arboreto) |
| **ATAC layer** | Peak co-accessibility (which regulatory regions are co-active) | CIRCE |
| **TF layer** | TF-TF co-expression | Correlation |
| **TF → ATAC** | Where TFs bind DNA (motif-based) | CellOracle motif scanning |
| **ATAC → RNA** | Which peaks regulate which genes | CIRCE peak-gene links |

```{note}
**Why multiple layers?**

GRNBoost2 alone infers statistical associations between TFs and genes, but these can include indirect relationships. By integrating chromatin accessibility (ATAC), we add mechanistic evidence: a TF can only regulate a gene if (1) it binds nearby accessible DNA and (2) that region is linked to the gene. The multilayer RWR combines both direct (TF→gene) and indirect (TF→ATAC→gene) evidence for more robust GRN inference.
```

### Step 1: Load TF list

In [6]:
# Load curated list of human transcription factors
tfs_list = hummuspy.loader.load_tfs("human_tfs_r_hummus")
print(f"Loaded {len(tfs_list)} transcription factors")

Loaded 914 transcription factors


### Step 2: Compute RNA network (gene co-regulation)

Uses GRNBoost2 to identify genes that co-vary with each TF. This captures statistical associations that may include both direct targets and genes in the same regulatory programs.

```{warning}
GRNBoost2 edges are used to represent **co-regulated** genes, not final TF-gene regulation. The multilayer integration in the final step will leverage these by requiring chromatin accessibility evidence.
```

In [7]:
# Compute TF → gene network using GRNBoost2
rna_network = recon.infer_grn.compute_rna_network(rna, tf_names=tfs_list, n_cpu=15)
print(f"RNA network: {len(rna_network)} edges")

Calculating TF-to-gene importance


Running using 15 cores:   0%|          | 0/1000 [00:00<?, ?it/s]

Running using 15 cores: 100%|██████████| 1000/1000 [00:03<00:00, 251.74it/s]


RNA network: 25881 edges


### Step 3: Compute ATAC network (peak co-accessibility)

Uses CIRCE to compute co-accessibility between ATAC-seq peaks.

In [8]:
# Prepare ATAC peak names (replace special characters)
atac.var_names = atac.var_names.str.replace("-", "_")
atac.var_names = atac.var_names.str.replace(":", "_")

# Add genomic region information
atac = ci.add_region_infos(atac)

# Compute peak co-accessibility network
ci.compute_atac_network(atac, njobs=20)

# Extract and format the network
atac_network = ci.extract_atac_links(atac, key='atac_network')
atac_network = atac_network.rename(columns={
    "Peak1": "source",
    "Peak2": "target",
    "score": "weight"
})
print(f"ATAC network: {len(atac_network)} edges")

Calculating co-accessibility scores...
Concatenating results from all chromosomes...
ATAC network: 172271 edges


### Step 4: Compute TF network (TF-TF interactions)

In [9]:
# Compute TF-TF correlation network
tf_network = recon.infer_grn.compute_tf_network(rna, tfs_list=tfs_list)
print(f"TF network: {len(tf_network)} edges")

TF network: 29 edges


In [10]:
tf_network.head(3)

,source,target
0,TFAP2E_TF,fake_TF
1,HES2_TF,fake_TF
2,ID3_TF,fake_TF


### Step 5: Compute bipartite links

Now we connect the layers:
- **ATAC → RNA**: Which peaks regulate which genes (based on proximity and correlation)
- **TF → ATAC**: Where TFs bind DNA (based on motif scanning)

```{note}
These steps require a **reference genome**. Make sure to install it first:
```python
import genomepy
genomepy.install_genome("hg19", "UCSC", annotation=True)
```
```

In [11]:
# ATAC → RNA: Peak-gene links based on proximity and correlation
atac_rna_links = recon.infer_grn.compute_atac_to_rna_links(atac, rna, ref_genome="hg19")
print(f"ATAC→RNA links: {len(atac_rna_links)} edges")

que bed peaks: 5000
tss peaks in que: 180
ATAC→RNA links: 130 edges


In [12]:
# TF → ATAC: Motif scanning to find TF binding sites
tf_atac_links = recon.infer_grn.compute_tf_to_atac_links(atac, ref_genome="hg19", n_cpus=30)
print(f"TF→ATAC links: {len(tf_atac_links)} edges")

Peaks before filtering:  5000
Peaks with invalid chr_name:  0
Peaks with invalid length:  0
Peaks after filtering:  5000
No motif data entered. Loading default motifs for your species ...
 Default motif for vertebrate: gimme.vertebrate.v5.0. 
 For more information, please see https://gimmemotifs.readthedocs.io/en/master/overview.html 

Initiating scanner... 



2026-01-12 14:15:29,884 - DEBUG - using background: genome hg19 with size 200


Calculating FPR-based threshold. This step may take substantial time when you load a new ref-genome. It will be done quicker on the second time. 

Motif scan started .. It may take long time.



Scanning: 100%|██████████| 5000/5000 [00:25<00:00, 196.45 sequences/s] 


Filtering finished: 702652 -> 156632
1. Converting scanned results into one-hot encoded dataframe.


100%|██████████| 4962/4962 [00:01<00:00, 3358.27it/s]


2. Converting results into dictionaries.


100%|██████████| 1093/1093 [00:00<00:00, 35187.82it/s]


TF→ATAC links: 2999856 edges


In [13]:
# Filter to keep only links from known TFs
tf_atac_links = tf_atac_links[tf_atac_links['source'].str.replace("_TF", "").isin(tfs_list)]
print(f"Filtered TF→ATAC links: {len(tf_atac_links)} edges")

Filtered TF→ATAC links: 1366039 edges


In [14]:
tf_atac_links.head(3)

,source,target
0,NEUROD1_TF,chr1_1000608_1001108
1,BHLHA15_TF,chr1_1000608_1001108
2,NEUROD2_TF,chr1_1000608_1001108


---

## Integrate into Final GRN

Now we combine all layers using Random Walk with Restart (RWR) to propagate information across the multilayer network.

```{tip}
The RWR integration weighs indirect evidence (TF → ATAC → RNA) alongside direct evidence (TF → RNA), improving GRN quality compared to single-layer approaches.
```

In [15]:
# Generate integrated GRN via RWR across all layers
grn = recon.infer_grn.generate_grn(
    rna_network=rna_network,
    atac_network=atac_network,
    tf_network=tf_network,
    atac_to_rna_links=atac_rna_links,
    tf_to_atac_links=tf_atac_links,
    n_jobs=20
)

# Keep only RNA layer (TF → gene relationships)
grn = grn[grn.layer == 'RNA']
print(f"Final GRN: {len(grn)} TF → gene edges")

TF_0
ATAC_0
RNA_0
Initializing Dask cluster with 20 workers...
http://192.168.152.25:8787/status


Running RWR per seed: 100%|██████████| 29/29 [00:01<00:00, 15.99it/s]


Building Dask DataFrame graph with delayed parallel tasks...
Final GRN: 2929 TF → gene edges


In [16]:
# Preview the final GRN
grn.head(10)

,index,layer,target,path_layer,score,seed
11262,880,RNA,TMEM35B,RNA_0,0.000106,TAL1_TF
11282,439,RNA,LINC01355,RNA_0,0.000105,TAL1_TF
11861,656,RNA,PLEKHG5,RNA_0,0.000102,NFYC_TF
15225,815,RNA,SPEN,RNA_0,0.000074,TP73_TF
15239,139,RNA,CELA2B,RNA_0,0.000074,TP73_TF
15280,6,RNA,ACOT7,RNA_0,0.000073,TP73_TF
15450,393,RNA,KAZN,RNA_0,0.000064,GFI1_TF
15511,958,RNA,XKR8,RNA_0,0.000056,ZNF691_TF
15520,641,RNA,PIGV,RNA_0,0.000054,ZNF691_TF
15521,117,RNA,CCDC27,RNA_0,0.000054,ZNF691_TF


---

## Save the GRN

Save the inferred GRN for use in other ReCoN tutorials (e.g., Tutorial 2: Multicellular Coordination).

In [17]:
# Save to CSV
grn.to_csv("recon_hummus_grn.csv", index=False)
print("GRN saved to recon_hummus_grn.csv")

GRN saved to recon_hummus_grn.csv


```{tip}
**Next steps**

Use this GRN in:
- [Tutorial 2: Multicellular Coordination](2.recon_multicellular_coordination.ipynb) - Explore upstream regulators
- [Tutorial 3: Molecular Cascades](3.recon_molecular_cascades.ipynb) - Visualize signaling pathways
```

---

This work uses **ReCoN**’s wrapper of different packages, which are all cited in the Methods section:

- The **RNA layer** uses [**arboreto¹**](#arboreto)  
- The **ATAC layer** uses [**CIRCE²**](#circe)  
- The **TF-ATAC** and **ATAC-RNA bipartites** use [**CellOracle³**](#celloracle)  
- The exploration of the **multilayer** uses [**HuMMuS⁴**](#hummus) and [**MultiXrank⁵**](#multixrank)  

---

### References
<a id="arboreto"></a>
[1] Moerman, T., Aibar, S., Bravo González-Blas, C., Simm, J., Moreau, Y., Aerts, J., & Aerts, S. (2019).  
**GRNBoost2 and Arboreto: efficient and scalable inference of gene regulatory networks.**  
*Bioinformatics*, 35(12), 2159–2161. doi:[10.1093/bioinformatics/bty916](https://doi.org/10.1093/bioinformatics/bty916)

<a id="circe"></a>
[2] Trimbour, R., Saez Rodriguez J., Cantini L. (2025).
**Circe: Co-accessibility network from ATAC-seq data in Python (software).**  
Version 0.3.6. [GitHub](https://github.com/cantinilab/Circe)

<a id="celloracle"></a>
[3] Kamimoto, K., Stringa, B., Hoffmann, C. M., Jindal, K., Solnica-Krezel, L., Morris, S. A., et al. (2023).  
**Dissecting cell identity via network inference and in silico gene perturbation.**  
*Nature*, 614, 742–751. doi:[10.1038/s41586-022-05688-9](https://doi.org/10.1038/s41586-022-05688-9)

<a id="hummus"></a>
[4] Trimbour, R., Deutschmann I. M., Cantini L. (2024).  
**Molecular mechanisms reconstruction from single-cell multi-omics data with HuMMuS.**  
*Bioinformatics*, 40(5), btae143. doi:[10.1093/bioinformatics/btae143](https://doi.org/10.1093/bioinformatics/btae143)

<a id="multixrank"></a>
[5] Baptista, A., González, A., & Baudot, A. (2022).  
**Universal multilayer network exploration by random walk with restart.**  
*Communications Physics*, 5, 170. doi:[10.1038/s42005-022-00937-9](https://doi.org/10.1038/s42005-022-00937-9)
